In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
train_data = [("me gusta comer en la cafeteria".split(), "SPANISH"),
        ("Give it to me".split(), "ENGLISH"),
        ("No creo que sea una buena idea".split(), "SPANISH"),
        ("No it is not a good idea to get lost at sea".split(), "ENGLISH")]

test_data = [("Yo creo que si".split(), "SPANISH"),
             ("it is lost on me".split(), "ENGLISH")]


In [3]:
from pprint import pprint

In [4]:
word_to_idx={}
for feat,label in train_data+test_data:
    for word in feat:
        if  word not in word_to_idx:
            word_to_idx[word]=len(word_to_idx)

pprint(sorted(word_to_idx.items(),key=lambda x : x[1]), indent=4)

VOCAB_SIZE = len(word_to_idx)
NUM_LABELS = 2


[   ('me', 0),
    ('gusta', 1),
    ('comer', 2),
    ('en', 3),
    ('la', 4),
    ('cafeteria', 5),
    ('Give', 6),
    ('it', 7),
    ('to', 8),
    ('No', 9),
    ('creo', 10),
    ('que', 11),
    ('sea', 12),
    ('una', 13),
    ('buena', 14),
    ('idea', 15),
    ('is', 16),
    ('not', 17),
    ('a', 18),
    ('good', 19),
    ('get', 20),
    ('lost', 21),
    ('at', 22),
    ('Yo', 23),
    ('si', 24),
    ('on', 25)]


In [5]:
class BoWClassifier(nn.Module):
    def __init__(self,num_labels,vocab_size):
        #or put it this way 
        #super(BowClassifier,self).__init__()
        super().__init__() 

        self.linear = nn.Linear(vocab_size,num_labels)

    def forward(self,bow_vec):
        #data points in form of row vectors
        return F.log_softmax(self.linear(bow_vec),1) 



In [6]:
def make_bow_vector(sentence, word_to_idx):

    vec = torch.zeros(len(word_to_idx))
    
    for word in sentence:
        #one hot encoding vector
        vec[word_to_idx[word]]+=1

    return vec.view(1,-1)

In [7]:
def make_target(label, label_to_idx):
    return torch.LongTensor([label_to_idx[label]])

In [8]:
model = BoWClassifier(NUM_LABELS,VOCAB_SIZE)

In [9]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[ 0.1011, -0.0866, -0.0380,  0.0921, -0.1846,  0.1176, -0.0403,  0.0998,
          0.0273, -0.0240,  0.0544,  0.0097,  0.0716, -0.0764, -0.0143, -0.0177,
          0.0284, -0.0008,  0.1714,  0.0610, -0.0730, -0.1184, -0.0329, -0.0846,
         -0.0628,  0.0094],
        [ 0.1169,  0.1066, -0.1917,  0.1216,  0.0548,  0.1860,  0.1294, -0.1787,
         -0.1865, -0.0946,  0.1722, -0.0327,  0.0839, -0.0911,  0.1924, -0.0830,
          0.1471,  0.0023, -0.1033,  0.1008, -0.1041,  0.0577, -0.0566, -0.0215,
         -0.1885, -0.0935]], requires_grad=True)
Parameter containing:
tensor([ 0.1064, -0.0477], requires_grad=True)


In [11]:
with torch.no_grad():
    sample = train_data[0]
    print(sample[0])
    bow_vector = make_bow_vector(sample[0], word_to_idx)
    pprint(bow_vector)
    log_probs= model(bow_vector)
    pprint(log_probs)

['me', 'gusta', 'comer', 'en', 'la', 'cafeteria']
tensor([[1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[-0.8195, -0.5810]])


In [15]:
label_to_idx = {"SPANISH": 0, "ENGLISH": 1}


In [16]:
#before training .
with torch.no_grad():
    for feat, label in test_data:
        bow_vec = make_bow_vector(feat, word_to_idx)
        log_probs = model(bow_vec)
        print(log_probs)

tensor([[-0.6250, -0.7662]])
tensor([[-0.5870, -0.8119]])


In [17]:
print(next(model.parameters())[: ,word_to_idx['buena']])


tensor([-0.0143,  0.1924], grad_fn=<SelectBackward>)


In [18]:
#hyperparams
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [19]:
for epoch in range(100):
    for instance,label in train_data:

        #pytorch needs to clean up the left over gradients 
        model.zero_grad()

        #making one hot word embedding vectors and string labels into ints
        bow_vec = make_bow_vector(instance, word_to_idx)
        target = make_target(label,label_to_idx)

        #forward pass
        log_probs = model(bow_vec)

        #computing loss fn based on predicted and target
        loss = loss_function(log_probs,target)
        loss.backward()
        optimizer.step()

In [20]:
with torch.no_grad():
    for feat, label in test_data:
        bow_vec = make_bow_vector(feat,word_to_idx)
        log_probs = model(bow_vec)
        print(log_probs)

tensor([[-0.1210, -2.1721]])
tensor([[-2.7767, -0.0643]])


In [21]:
print(next(model.parameters())[: ,word_to_idx['buena']])
#this output shows improvement in the suggestion that this belongs to spanish indeed

tensor([ 0.4318, -0.2536], grad_fn=<SelectBackward>)
